# Variography

In this preamble, we load the **gstlearn** library.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np
import os
import urllib.request

Then the necessary data set is downloaded and named **dat**: the target variable is **January_temp**

In [ ]:
url = 'https://soft.minesparis.psl.eu/gstlearn/data/Scotland/Scotland_Temperatures.NF'
temp_nf, head = urllib.request.urlretrieve(url)
dat = gl.Db.createFromNF(temp_nf)

## Experimental variogram

Data is a *regionalized* variable

$$z_i = z(x_i)$$

The experimental variogram is a (discrete) function:

$$\gamma(h)=\frac{1}{2N(h)}\sum_{i=1}^{N(h)}[z(x_i+h)-z(x_i)]^2$$

where $N(h)$ is the number of pairs of points distant by $h$



### Variogram Cloud

In [ ]:
varioParamOmni = gl.VarioParam.createOmniDirection(100)
grid_cloud = gl.db_variogram_cloud(dat, varioParamOmni)
grid_cloud

In [ ]:
ax = grid_cloud.plot("Cloud*")
plt.gca().set_aspect('100')
ax.decoration(title="Variogram Cloud")

### Experimental Variogram

We calculate the omni-directional variogram of the temperatures.

In [ ]:
varioParamOmni = gl.VarioParam.createOmniDirection(40, 10)
varioexp = gl.Vario(varioParamOmni, dat)
err = varioexp.compute()

Print the variogram contents

In [ ]:
varioexp

Plot the omni-directional variogram

In [ ]:
gp.setDefault(dims=[6,6])
gp.varmod(varioexp)

Represent the variogram with the number of pairs. By storing the *Axes* in a variable *ax* returned by the last instruction, we prevent jupyter from displaying '\<AxesSubplot:\>' before the plot.

In [ ]:
ax = gp.varmod(varioexp,show_pairs=True)

Automatic Fitting procedure (with default fitting parameters).

In [ ]:
fitmod = gl.Model()
err = fitmod.fit(varioexp)
gp.varmod(varioexp, fitmod)
plt.show() # This last instruction also prevents jupyter from displaying a text before the plot

Print the Fittied Model content

In [ ]:
fitmod

List of Basic Covariances structures known by **gstlearn**

In [ ]:
gl.ECov.printAll()

Automatic Fitting (using given basic structures)

In [ ]:
types = [gl.ECov.NUGGET, gl.ECov.CUBIC, gl.ECov.SPHERICAL]
err = fitmod.fit(varioexp, types=types)
ax = gp.varmod(varioexp, fitmod)

The resulting Model

In [ ]:
fitmod

Model Fitting with Inequality constraints

In [ ]:
constraints = gl.Constraints()
err = constraints.addItemFromParamId(gl.EConsElem.RANGE,1,0,0,gl.EConsType.UPPER,20.)
err = constraints.addItemFromParamId(gl.EConsElem.SILL,1,0,0,gl.EConsType.LOWER,0.03)
err = fitmod.fit(varioexp, types, constraints, gl.Option_VarioFit(True))
ax = gp.varmod(varioexp, fitmod)

The resulting Model

In [ ]:
fitmod

Model Fitting with Equality constraints

In [ ]:
constraints = gl.Constraints()
err = constraints.addItemFromParamId(gl.EConsElem.RANGE,1,0,0,gl.EConsType.EQUAL,1000.)
err = constraints.addItemFromParamId(gl.EConsElem.SILL,1,0,0,gl.EConsType.EQUAL,0.4)
err = fitmod.fit(varioexp, types, constraints, gl.Option_VarioFit(True))
ax = gp.varmod(varioexp, fitmod)

The resulting Model

In [ ]:
fitmod

Directional Variograms

In [ ]:
varioParamMulti = gl.VarioParam.createMultiple(4, 15, 15.)
vario_4dir = gl.Vario(varioParamMulti, dat)
err = vario_4dir.compute()
ax = gp.varmod(vario_4dir, flagLegend=True)

Fitting a Multi-directional variogram

In [ ]:
model_4dir = gl.Model()
err = model_4dir.fit(vario_4dir,types=types)
ax = gp.varmod(vario_4dir, model_4dir)

Calculating Variogram Map (the result is returned as a Grid data base displayed using *raster* function). At the right is displayed the number of pairs used for the variogram calculation in each grid cell.

In [ ]:
grid_vmap = gl.db_vmap_compute(dat, gl.ECalcVario.VARIOGRAM)
grid_vmap.display()
fig, ax = plt.subplots(1,2,figsize=[14,10])
fig.tight_layout(pad=5.0)
ax[0].raster(grid_vmap, flagLegend=True)
ax[1].raster(grid_vmap, name="*Nb", flagLegend=True)
plt.show()

Automatic Model Fitting from Variogram Map

In [ ]:
modelVM = gl.Model()
err = modelVM.fitFromVMap(grid_vmap, types=types)
modelVM

Drawing the Fitted Model as a Variogram Map

In [ ]:
err = gl.dbgrid_model(grid_vmap, modelVM)
grid_vmap.display()
ax = gp.raster(grid_vmap, flagLegend=True)

Compare Directional Variograms and fitted model from Variogram Map

In [ ]:
ax = gp.varmod(vario_4dir, modelVM)